# Estimagic with SciPy's `least_squares`

This is the accompanying documentation to the Pull Request adding the scipy [least_squares](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html) to [estimagic](https://github.com/OpenSourceEconomics/estimagic).



## Estimagic
[estimagic](https://github.com/OpenSourceEconomics/estimagic) is a Python package that provides tools for nonlinear optimization, numerical differentiation and statistical inference to fit large scale empirical models to data and make inferences about the estimated model parameters. It is well suited to solving difficult constrained optimization problems.

The full documentation can be accessed [here](https://estimagic.readthedocs.io/en/latest/#).

estimagic provides a large collection of [internal optimizers](https://estimagic.readthedocs.io/en/latest/explanations/optimization/internal_optimizers.html). In this project, I add three new algorithms from [scipy.optimize.least_squares](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html#r20fc1df64af7-stir).

## Steps taken
To include the optimizer, I conducted the following steps:

1. Defined a new function `_scipy_least_squares`, where I specified all parameters and ran all the tests successfully.
2. In order to keep the convention used in estimagic, I created three functions `scipy_ls_trf`, `scipy_ls_dogbox` and `scipy_ls_lm` corresponding to the three different algorithms used in `_scipy_least_squares` by calling it and soecifying the correct method argument. 
3. Fix the broken link `https://gitlab.com/petsc/petsc/-/tree/master/src/binding/petsc4py/`, so the documentation builds

## Algorithms
The `scipy.optimize.least_sqaures` solves a nonlinear least-squares problem with bounds on the variables. It uses three alrogithms, which are in turn added to estimagic:

### 1. Trust Region Reflective `scipy_ls_trf`
Ths method is motivated by the process of solving a system of equations, which constitute the first-order optimality condition for a bound-constrained minimization problem as formulated in (reference paper). The algorithm iteratively solves trust-region subproblems augmented by a special diagonal quadratic term and with trust-region shape determined by the distance from the bounds and the direction of the gradient. This enhancements help to avoid making steps directly into bounds and efficiently explore the whole space of variables. To further improve convergence, the algorithm considers search directions reflected from the bounds. To obey theoretical requirements, the algorithm keeps iterates strictly feasible. With dense Jacobians trust-region subproblems are solved by an exact method very similar to the one described in (reference another paper) (and implemented in MINPACK). The difference from the MINPACK implementation is that a singular value decomposition of a Jacobian matrix is done once per iteration, instead of a QR decomposition and series of Givens rotation eliminations. For large sparse Jacobians a 2-D subspace approach of solving trust-region subproblems is used (reference)]. The subspace is spanned by a scaled gradient and an approximate Gauss-Newton solution delivered by `scipy.sparse.linalg.lsmr`. When no constraints are imposed the algorithm is very similar to MINPACK and has generally comparable performance. The algorithm works quite robust in unbounded and bounded problems, thus it is chosen as a default algorithm.
The documentation can be found here. 

### 2. DogBox `scipy_ls_dogbox`
This method operates in a trust-region framework, but considers rectangular trust regions as opposed to conventional ellipsoids (reference the paper). The intersection of a current trust region and initial bounds is again rectangular, so on each iteration a quadratic minimization problem subject to bound constraints is solved approximately by Powell’s dogleg method (use reference). The required Gauss-Newton step can be computed exactly for dense Jacobians or approximately by `scipy.sparse.linalg.lsmr` for large sparse Jacobians. The algorithm is likely to exhibit slow convergence when the rank of Jacobian is less than the number of variables. The algorithm often outperforms `scipy_ls_trf` in bounded problems with a small number of variables.

### 3. Levenberg-Marquardt `scipy_ls_lm`
This method calls a wrapper over least-squares algorithms implemented in MINPACK (lmder, lmdif). It runs the Levenberg-Marquardt algorithm formulated as a trust-region type algorithm. The implementation is based on paper cite the paper, it is very robust and efficient with a lot of smart tricks. It should be your first choice for unconstrained problems. Note that it doesn’t support bounds. Also, it doesn’t work when m < n.

## Sphere Example
To show that the optimizer works within estimagic, I've included the [Sphere Example](https://estimagic.readthedocs.io/en/latest/getting_started/first_optimization_with_estimagic.html) from the documentation.


In [1]:
import numpy as np
import pandas as pd
from estimagic import minimize
from estimagic.logging.read_log import read_optimization_iteration

/home/yana/PycharmProjects/estimagic/venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def sphere(params):
    """Spherical criterion function.

    The unique local and global optimum of this function is at
    the zero vector. It is differentiable, convex and extremely
    well behaved in any possible sense.

    Args:
        params (pandas.DataFrame): DataFrame with the columns
            "value", "lower_bound", "upper_bound" and potentially more.

    Returns:
        dict: A dictionary with the entries "value" and "root_contributions".

    """
    out = {
        "value": (params["value"] ** 2).sum(),
        "root_contributions": params["value"],
    }
    return out


def sphere_gradient(params):
    """Gradient of spherical criterion function"""
    return params["value"] * 2

In [3]:
start_params = pd.DataFrame(
    data=np.arange(5) + 1,
    columns=["value"],
    index=[f"x_{i}" for i in range(5)],
)
params_with_bounds = start_params.copy()

params_with_bounds["lower_bound"] = [0, 1, 0, -1, 0]
params_with_bounds["upper_bound"] = [np.inf] * 5


In [4]:
# With bounds
res = minimize(
    criterion=sphere,
    params=params_with_bounds,
    algorithm="scipy_least_squares",
    derivative=sphere_gradient
)
res["solution_params"].round(2)

,lower_bound,upper_bound,value
x_0,0.0,inf,0.0
x_1,1.0,inf,1.0
x_2,0.0,inf,0.0
x_3,-1.0,inf,-0.0
x_4,0.0,inf,0.0


In [5]:
# With bounds and constraints
constraints = [{"loc": ["x_0", "x_3"], "type": "fixed", "value": [1, 4]}]
res = minimize(
    criterion=sphere,
    params=params_with_bounds,
    algorithm="scipy_least_squares",
    derivative=sphere_gradient,
    constraints=constraints,
)
res["solution_params"].round(2)

,lower_bound,upper_bound,value
x_0,0.0,inf,1.0
x_1,1.0,inf,1.0
x_2,0.0,inf,0.0
x_3,-1.0,inf,4.0
x_4,0.0,inf,0.0


## Supplying Arguments
The included `least_squares` optimizer takes arguments (see documentation).
This example sets the internal algorithm to `dogbox`.

In [6]:
# Use the dogbox algorithm instead of the default
algo_options = {
    'method': 'dogbox'
}

res = minimize(
    criterion=sphere,
    params=start_params,
    algorithm="scipy_least_squares",
    derivative=sphere_gradient,
    algo_options=algo_options
)
res["solution_params"].round(2)

,lower_bound,upper_bound,value
x_0,-inf,inf,0.0
x_1,-inf,inf,0.0
x_2,-inf,inf,0.0
x_3,-inf,inf,0.0
x_4,-inf,inf,0.0


## Rosenbrock Example
To further illustrate the optimizers capabilities, I've also included the Rosenbrock example from the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html).

### The Rosenbrock Function
In mathematical optimization, the Rosenbrock function is a non-convex function, introduced by Howard H. Rosenbrock in 1960, which is used as a performance test problem for optimization algorithms. It is also known as Rosenbrock's valley or Rosenbrock's banana function. [[Rosenbrock]](https://academic.oup.com/comjnl/article/3/3/175/345501)

In [7]:
def fun_rosenbrock(params):
    x = params["value"]
    res = 10 * (x[1] - x[0]**2)**2 + (1 - x[0])**2
    return {
        "value": res,
        "root_contributions": x
    }
def jac_rosenbrock(params):
    x, y = params["value"]
    return [
        2 * (20 * x**3 - 20 * x * y + x - 1), # diff. by x
        20 * (y - x**2)  # diff. by y
    ]

In [8]:
x0_rosenbrock = pd.DataFrame(
    data=[2, 2],
    columns=["value"],
    index=[f"x_{i}" for i in range(2)],
)

In [9]:
# Without bounds
res = minimize(
    criterion=fun_rosenbrock,
    params=x0_rosenbrock,
    algorithm="scipy_least_squares",
    derivative=jac_rosenbrock,
)
# Function has minima at (1, 1)
res["solution_params"].round(2)

,lower_bound,upper_bound,value
x_0,-inf,inf,1.0
x_1,-inf,inf,1.0


In [ ]:
# With bounds
x0_rosenbrock["lower_bound"] = [-np.inf, 1.5]
x0_rosenbrock["upper_bound"] = [np.inf] * 2
res = minimize(
    criterion=fun_rosenbrock,
    params=x0_rosenbrock,
    algorithm="scipy_least_squares",
    derivative=jac_rosenbrock,
)
# Function has new solution that lies on the bound (1.22, 1.5)
res["solution_params"].round(2)